# Wine Quality Prediction - Part 1 - Regression

![](https://cdn.pixabay.com/photo/2016/03/09/11/53/wine-glasses-1246240_1280.jpg)

## Introduction

This notebook is part of a trilogy in which I will approach the wine quality dataset from several different approaches:

+ [Part 1: Supervised Learning - Regression](https://www.kaggle.com/sgtsteiner/red-wine-quality-regression)
+ [Part 2: Supervised Learning - Multiclass Classification](https://www.kaggle.com/sgtsteiner/red-wine-quality-multiclass-classification)
+ [Part 3: Supervised Learning - Binary Classification](https://www.kaggle.com/sgtsteiner/red-wine-quality-binary-classification)

## Frame the problem

We have a dataset that contains various characteristics of red and white variants of the Portuguese "Vinho Verde" wine. We have chemical variables, such as the amount of alcohol, citric acid, acidity, density, pH, etc; as well as a sensorial and subjective variable such as the score with which a group of experts rated the quality of the wine: between 0 (very bad) and 10 (very excellent).

They ask us to build a model that can predict the quality score given these biochemical indicators.

For this first part of the study, we are going to consider that it is a **regression problem**.

### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_validate, StratifiedKFold
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, ElasticNet, Ridge
from sklearn.feature_selection import RFE, RFECV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn import metrics


%matplotlib inline

## Get the Data

In [ ]:
red = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

### Check the size and type of data

In [ ]:
red.shape

In [ ]:
red.head()

In [ ]:
red.info()

In [ ]:
pd.DataFrame({"Type": red.dtypes,
              "Unique": red.nunique(),
              "Null": red.isnull().sum(),
              "Null percent": red.isnull().sum() / len(red),
              "Mean": red.mean(),
              "Std": red.std()})

Mmmmm, there are no nulls, what a data set!

In [ ]:
red.describe().T

## Explore the Data

How are the features distributed?

In [ ]:
red.hist(bins=50, figsize=(15,12));

Let's check how our target variable, the quality score, is distributed:

In [ ]:
print(f"Percentage of quality scores")
red["quality"].value_counts(normalize=True) * 100

It is significantly unbalanced. Most instances (82%) have scores of 6 or 5.

We are going to check the correlations between the attributes of the dataset:

In [ ]:
corr_matrix = red.corr()
corr_matrix

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(red.corr(), annot=True, cmap='coolwarm')
plt.show()

We show only the correlations of the target variable with the rest of the attributes:

In [ ]:
corr_matrix["quality"].drop("quality").sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(8,5))
corr_matrix["quality"].drop("quality").sort_values(ascending=False).plot(kind='bar')
plt.title("Attribute correlations with quality")
plt.show()

## Prepare the Data

Create the predictor set and the set with the target variable:

In [ ]:
predict_columns = red.columns[:-1]
predict_columns

In [ ]:
X = red[predict_columns]
y = red["quality"]

Create the training and test datasets:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

## Shortlist Promising Models

OK, we're going train several quick-and-dirty models from different categories using standard parameters. We selected some of the regression models: Linear Regression, Lasso, ElasticNet, Ridge, Extre Trees, and RandomForest.

In [ ]:
def evaluate_model(estimator, X_train, y_train, cv=10, verbose=True):
    """Print and return cross validation of model
    """
    scoring = ["neg_mean_absolute_error", "neg_mean_squared_error", "r2"]
    scores = cross_validate(estimator, X_train, y_train, return_train_score=True, cv=cv, scoring=scoring)
    
    val_mae_mean, val_mae_std = -scores['test_neg_mean_absolute_error'].mean(), \
                                -scores['test_neg_mean_absolute_error'].std()
    
    train_mae_mean, train_mae_std = -scores['train_neg_mean_absolute_error'].mean(), \
                                    -scores['train_neg_mean_absolute_error'].std()
    
    val_mse_mean, val_mse_std = -scores['test_neg_mean_squared_error'].mean(), \
                                -scores['test_neg_mean_squared_error'].std()
    
    train_mse_mean, train_mse_std = -scores['train_neg_mean_squared_error'].mean(), \
                                    -scores['train_neg_mean_squared_error'].std()
    
    val_rmse_mean, val_rmse_std = np.sqrt(-scores['test_neg_mean_squared_error']).mean(), \
                                  np.sqrt(-scores['test_neg_mean_squared_error']).std()
    
    train_rmse_mean, train_rmse_std = np.sqrt(-scores['train_neg_mean_squared_error']).mean(), \
                                      np.sqrt(-scores['train_neg_mean_squared_error']).std()
    
    val_r2_mean, val_r2_std = scores['test_r2'].mean(), scores['test_r2'].std()
    
    train_r2_mean, train_r2_std = scores['train_r2'].mean(), scores['train_r2'].std()

    
    result = {
        "Val MAE": val_mae_mean,
        "Val MAE std": val_mae_std,
        "Train MAE": train_mae_mean,
        "Train MAE std": train_mae_std,
        "Val MSE": val_mse_mean,
        "Val MSE std": val_mse_std,
        "Train MSE": train_mse_mean,
        "Train MSE std": train_mse_std,
        "Val RMSE": val_rmse_mean,
        "Val RMSE std": val_rmse_std,
        "Train RMSE": train_rmse_mean,
        "Train RMSE std": train_rmse_std,
        "Val R2": val_r2_mean,
        "Val R2 std": val_r2_std,
        "Train R2": train_rmse_mean,
        "Train R2 std": train_r2_std,
    }
    
    if verbose:
        print(f"val_MAE_mean: {val_mae_mean} - (std: {val_mae_std})")
        print(f"train_MAE_mean: {train_mae_mean} - (std: {train_mae_std})")
        print(f"val_MSE_mean: {val_mse_mean} - (std: {val_mse_std})")
        print(f"train_MSE_mean: {train_mse_mean} - (std: {train_mse_std})")
        print(f"val_RMSE_mean: {val_rmse_mean} - (std: {val_rmse_std})")
        print(f"train_RMSE_mean: {train_rmse_mean} - (std: {train_rmse_std})")
        print(f"val_R2_mean: {val_r2_mean} - (std: {val_r2_std})")
        print(f"train_R2_mean: {train_r2_mean} - (std: {train_r2_std})")

    return result

In [ ]:
models = [LinearRegression(), Lasso(alpha=0.1), ElasticNet(),
          Ridge(), ExtraTreesRegressor(), RandomForestRegressor()]

model_names = ["Lineal Regression", "Lasso", "ElasticNet",
               "Ridge", "Extra Tree", "Random Forest"]

In [ ]:
mae = []
mse = []
rmse = []
r2 = []

for model in range(len(models)):
    print(f"Paso {model+1} de {len(models)}")
    print(f"...running {model_names[model]}")
    
    rg_scores = evaluate_model(models[model], X_train, y_train)
    
    mae.append(rg_scores["Val MAE"])
    mse.append(rg_scores["Val MSE"])
    rmse.append(rg_scores["Val RMSE"])
    r2.append(rg_scores["Val R2"])

Let's see the performance of each of them:

In [ ]:
df_result = pd.DataFrame({"Model": model_names,
                          "MAE": mae,
                          "MSE": mse,
                          "RMSE": rmse,
                          "R2": r2})
df_result

In [ ]:
df_result.sort_values(by="RMSE", ascending=False).plot.barh("Model", "RMSE");

In [ ]:
df_result.sort_values(by="R2").plot.barh("Model", "R2");

The model that gives the best results is **extra trees**. RMSE = 0.577591 and R2 = 0.477845. Let's fine tune it.

## Fine-Tune

In [ ]:
param_grid = [
    {'n_estimators': range(10, 300, 10), 'max_features': [2, 3, 4, 5, 8, "auto"], 'bootstrap': [True, False]}
]


xtree_reg = ExtraTreesRegressor(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(xtree_reg, param_grid, cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

It's the moment of truth! Let's see the performance on the test set:

In [ ]:
final_model = grid_search.best_estimator_
y_pred = final_model.predict(X_test)

In [ ]:
print(f"MAE: {metrics.mean_absolute_error(y_test, y_pred)}")
print(f"MSE: {metrics.mean_squared_error(y_test, y_pred)}")
print(f"RMSE: {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}")
print(f"R2: {final_model.score(X_test, y_test)}")

Well, a little better!

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(y_test, y_pred, alpha=0.1)
plt.xlabel("Real")
plt.ylabel("Predicted")
plt.show()

Let's see which features are most relevant:

In [ ]:
feature_importances = final_model.feature_importances_
feature_importances

In [ ]:
sorted(zip(feature_importances, X_test.columns), reverse=True)

In [ ]:
feature_imp = pd.Series(feature_importances, index=X_train.columns).sort_values(ascending=False)
feature_imp.plot(kind='bar')
plt.title('Feature Importances')

Let's see how the errors are distributed:

In [ ]:
df_resul = pd.DataFrame({"Pred": y_pred,
              "Real": y_test,
              "error": y_pred - y_test,
              "error_abs": abs(y_pred - y_test)})

In [ ]:
df_resul["error"].plot.hist(bins=40, density=True)
plt.title("Error distribution")
plt.xlabel("Error");

More generally, What's the MAE that occurs in each quality score?

In [ ]:
df_resul.groupby("Real")["error_abs"].mean()

In [ ]:
df_resul.groupby("Real")["error_abs"].mean().plot.bar()
plt.title("MAE distribution")
plt.ylabel("MAE")
plt.xlabel("Quality");

## Conclusions

After testing various models, the one that provided the best results is ExtraTrees. After fine tuning it, we get a significant improvement.

The basic line regression model offers an R2: 0.323021 and RMSE: 0.657899. The Extra Tree model offers an R2: 0.529512 and RMSE: 0.570954. However, the R2 score is still very low. According to the value obtained from R2, our model can barely explain 52% of the variance. That is, the percentage of relationship between the variables that can be explained by our model is 52.95%.

According to the MAE distribution graph, we can see that our model is not good for extreme scores. In fact, it is not capable of predicting any score of 3 or 8. As we saw in the distribution of the target variable, it is very unbalanced, there are hardly any observations for the extreme values, so the model does not have enough data training for all quality scores.

As a final consideration, we should try to approach modeling as a classification problem, to evaluate if it offers better results than a regression problem. We will see it in part 2 and 3 of this analysis.